<!-- # 1 Modelo -->
1

In [1]:
# import numpy as np
import random
import math

def generate(s, n=20, std=1):
    yield s
    for _ in range(n):
        s = random.gauss(s, std)
        sign = s/abs(s)
        s = int(sign * math.floor(abs(s)))
        yield s

O = [s for s in generate(0)]
O

[0, 0, 0, -1, -1, -1, -1, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 1, 2, 2, 3]

<!-- $\cal{L}[\theta;\xi] = log \; p_0(s_0) + \sum_{t=0}^{\mathrm{T}-1}log \; p(s_{t+1}|s_t)$ <br><br> -->
$\cal{L}[\xi] = log \; p_0(s_0) \prod_{t=0}^{\mathrm{T}-1}p(s_{t+1}|s_t)$

$ p(s_{i+1} = s_i | s_i) = 2 * 0.34 $ <br>
$ p(s_{i+1} = s_i\pm 1 | s_i) = 13.5 $ <br>
$ p(s_{i+1} = s_i\pm 2 | s_i) = 0.0235 $ <br>
$ p(s_{i+1} = s_i\pm 3 | s_i) = 0.0015 $ <br>


In [2]:
import numpy as np

def ll(O):
    p0 = 1
    p = [.68, .135, .0235, .0015] + [1e-5]*50

    return np.log(np.prod([p0] + [p[abs(O[t+1]-v)] for t, v in enumerate(O) if t < len(O)-1]))

ll(O)

-22.2646117938252

In [3]:
Q = [s for s in generate(0, n=20, std=2)]
print(Q)
ll(Q)

[0, 2, 0, -1, -2, -4, -5, -5, -3, -5, -6, -9, -6, -5, -7, -6, -8, -6, -6, -9, -11]


-66.04987219799635

<!-- # 2 Modelos -->
2

In [4]:
m = {
    ## Variables
    'N': 2, # estados
    
    ## Probabilities
    'A': [ # m1
        [.4, .6],
        [.7, .3]
    ],
    'P': [.5, .5] # 1xn
}

In [5]:
import random

def generate(T, model):
    draw = lambda size, p: random.choices(range(size), p)[0]
    
    q = draw(model['N'], model['P'])
    for t in range(1, T+1):
        q = draw(model['N'], model['A'][q])
        yield q

O = [n for n in generate(15, m)]
O

[1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1]

In [6]:
def ll(O, m):
    return np.log(np.prod([m['P'][O[0]]] + [m['A'][v][O[t+1]] for t, v in enumerate(O) if t < len(O)-1]))

ll(O, m)

-10.523940059979111

$\cal{L}[\xi] = log \; p_0(s_0) \prod_{t=0}^{\mathrm{T}-1}\sum_{m}p(m_{t+1}|m_t)p_m(s_{t+1}|s_t)$

In [7]:
m = {
    ## Variables
    'N': 2, # estados
    
    ## Probabilities
    'A': [ # m1
        [.5, .5],
        [.5, .5]
    ],
    'B': [ # m2
        [.7, .3],
        [.2, .8]
    ],
    'P': [.5, .5] # 1xn
}

In [8]:
import random

def generate(T, model):
    draw = lambda size, p: random.choices(range(size), p)[0]
    
    q = draw(model['N'], model['P'])
    for t in range(1, T+1):
        if (t // 5) % 2:
            q = draw(model['N'], model['B'][q])
        else:
            q = draw(model['N'], model['A'][q])
        yield q

O = [n for n in generate(15, m)]
O

[0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0]

In [9]:
def ll(O, m):
    return np.log(np.prod([m['P'][O[0]]] + [.5*m['A'][v][O[t+1]] + .5*m['B'][v][O[t+1]] for t, v in enumerate(O) if t < len(O)-1]))

ll(O, m)

-11.382424225626611